## PyRMS test script

simple test script for executing rms from python, for use in the uncertainty pipeline. rms does sensitivities faster


In [28]:
from pyrms import rms
from diffeqpy import de
from julia import Main
import yaml
from julia import Sundials
from diffeqpy import de
import time 
import matplotlib
from copy import deepcopy
from rmgpy.chemkin import load_chemkin_file
from rmgpy.rmg.model import ReactionModel
from rmgpy.species import Species
from rmgpy.kinetics import StickingCoefficientBEP, StickingCoefficient, SurfaceArrheniusBEP, SurfaceArrhenius
from rmgpy.data.kinetics.database import KineticsDatabase
import os
import copy
%matplotlib inline

In [29]:
file_dir = "/Users/blais.ch/Documents/_01_code/05_Project_repos_Github/meOH_repos/uncertainty_analysis/rmg_gua/baseline/rms/chem53.rms"
phase_dict = rms.readinput(file_dir)

In [30]:
base_path = "/".join(file_dir.split("/")[:-2])
base_path = os.path.join(base_path, "chemkin")
cmkn_path = os.path.join(base_path, "chem_annotated-gas.inp")
cmkn_surf_path = os.path.join(base_path, "chem_annotated-surface.inp")
cmkn_dict_path = os.path.join(base_path, "species_dictionary.txt")
# file_dir.replace(file_dir.split("/")[-1], "/chemkin")

In [31]:
expt_condts = "/Users/blais.ch/Documents/_01_code/05_Project_repos_Github/meOH_repos/uncertainty_analysis/rmg_gua/gua_cantera/all_experiments_reorg_sbr.yaml"

with open(expt_condts, 'r') as file:
    data = yaml.safe_load(file)

# pick just one experiment for example 
conditions = data[2]

# convert volume flow to molar flow
conditions["volume_flowrate"]

FC_temp = 293.15
conditions["molar_flow"] = conditions["volume_flowrate"] * 1.01325e5 / (8.3145 * FC_temp) 

In [32]:
intfc_key = list(phase_dict.keys())[0]

# mechanism dictionaries index:  phase_dict[phasename]["Species" or "Reactions"]
gasspcs = phase_dict["gas"]["Species"]
gasrxns = phase_dict["gas"]["Reactions"]
surfacespcs = phase_dict["surface"]["Species"]
surfacerxns = phase_dict["surface"]["Reactions"]
interfacerxns = phase_dict[intfc_key]["Reactions"]

In [33]:
#Define the phase (how species thermodynamic and kinetic properties calculated)
ig = rms.IdealGas(gasspcs,gasrxns,name="gas") 
cat = rms.IdealSurface(surfacespcs, surfacerxns, 2.943e-5, name="surface")

In [34]:
# Set simulation gas Initial Temp and Pressure
initialcondsgas = {
        "T":conditions["temperature"],
        "P":conditions["pressure"],
        "CO":conditions["species"]["CO"],
        "CO2":conditions["species"]["CO2"],
        "H2":conditions["species"]["H2"],
} 
# Define the domain (encodes how system thermodynamic properties calculated)
domaingas,y0gas,pgas = rms.ConstantTPDomain(phase=ig,initialconds=initialcondsgas,sensitivity=True)

In [35]:
# Set simulation surf Initial Temp and Pressure
V = conditions["volume"]
A = conditions["catalyst_area"]
initialconds = {
        "T":conditions["temperature"],
        "A":conditions["catalyst_area"],
        "X":cat.sitedensity*A
} 
# Define the domain (encodes how system thermodynamic properties calculated)
domaincat,y0cat,pcat = rms.ConstantTAPhiDomain(phase=cat,initialconds=initialconds,sensitivity=True);

## make reactor, inlet and outlet
- makes an anonymous function x->42, is that velocity in? need to check if it is velocity or volume flowrate
- also, I think the ```phi``` refers to chemical potential, but I should check, I think constantTPhi is just const T for our case. 

In [36]:
initialcondsinlet = {
        "T":conditions["temperature"],
        "P":conditions["pressure"],
        "CO":conditions["species"]["CO"],
        "CO2":conditions["species"]["CO2"],
        "H2":conditions["species"]["H2"],
    }

# construct reactor
inter,pinter = rms.ReactiveInternalInterfaceConstantTPhi(domaingas,domaincat,interfacerxns,initialcondsinlet["T"],A);

# make inlet and outlet
inletgas = rms.Inlet(domaingas,initialcondsinlet,Main.eval("x->"+str(conditions["molar_flow"])))
outletgas = rms.Outlet(domaingas,Main.eval("x->"+str(conditions["molar_flow"])))

In [37]:
# Define domains and interfaces
domains = (domaingas,domaincat)
interfaces = [inter,inletgas,outletgas]

# create a reactor for the system
react,y0,p = rms.Reactor(domains,(y0gas,y0cat),(0.0,100),interfaces,(pgas,pcat,pinter)) # Create the reactor object

In [38]:
# run the simulation
t1 = time.time()
sol = de.solve(react.ode,de.CVODE_BDF(),abstol=1e-20,reltol=1e-8)
t2 = time.time()
print("elapsed time for sim: ", t2-t1)

elapsed time for sim:  0.25319600105285645


In [39]:
ssys = rms.SystemSimulation(sol,domains,interfaces,p)

In [40]:
results = {}
gas_names = ssys.sims[0].names
gas_moles = rms.molefractions(ssys.sims[0], 100)
for (name, moles) in zip(gas_names, gas_moles):
    results[name] = moles

# surface
surf_names = ssys.sims[1].names
surf_moles = rms.molefractions(ssys.sims[1],  100)
for (name, moles) in zip(surf_names, surf_moles):
    results[name] = moles


In [41]:
len(gas_moles) ==len(gas_names)

True

In [42]:
results

{'N2': 0.0,
 'Ne': 0.0,
 'H2': 0.04164883545722103,
 'CO': 0.1000626484453466,
 'CO2': 0.13237922193777105,
 'H2O': 0.5386249256880566,
 'CH2O': 7.469821028233595e-11,
 'HCOOH': 3.725676036147179e-08,
 'CH3OH': 4.129487972930393e-07,
 'HCOOCH3': 3.482633031606602e-13,
 'CC(=O)O': 5.375587579443266e-08,
 'CCO': 8.388932811780068e-09,
 'CC=O': 4.0429323559552495e-10,
 'CC': 0.08082790603336504,
 'CH4': 0.10645594960853413,
 'X': 0.6162868580762426,
 'H*': 0.006040081325182351,
 'O*': 7.695385081732824e-09,
 'OH*': 9.586582536897821e-11,
 'H2O*': 0.02794832949289395,
 'CO*': 0.00029843758194912537,
 'CO2*': 0.035734537881668034,
 'HCO*': 8.351686310992187e-07,
 'HCOO*': 4.859161343805154e-05,
 'COOH*': 1.4116742462032423e-06,
 'HCOOH*': 3.888306812386699e-06,
 'CH2O*': 3.772078494818949e-08,
 'CH3O*': 2.029480818852885e-13,
 'CH3O2*': 3.1241916626860184e-16,
 'CH3OH*': 2.5011860810508693e-07,
 'OC[Pt]': 3.476696631820763e-07,
 'O=COC[Pt]': 2.1529639012023253e-14,
 'OCOC[Pt]': 2.6252179076

In [43]:
rms.molefractions(ssys.sims[0], "CO", 100)

0.1000626484453466

In [44]:
# load the chemkin file for the mechanism
chemkin_file = "/Users/blais.ch/Documents/_01_code/05_Project_repos_Github/meOH_repos/uncertainty_analysis/rmg_gua/baseline/chemkin/chem_annotated-gas.inp"
chemkin_surf_file = "/Users/blais.ch/Documents/_01_code/05_Project_repos_Github/meOH_repos/uncertainty_analysis/rmg_gua/baseline/chemkin/chem_annotated-surface.inp"
chemkin_dict = "/Users/blais.ch/Documents/_01_code/05_Project_repos_Github/meOH_repos/uncertainty_analysis/rmg_gua/baseline/chemkin/species_dictionary.txt"
model = ReactionModel()

model.species, model.reactions = load_chemkin_file(
    chemkin_file,
    chemkin_dict, 
    surface_path=chemkin_surf_file,
    )

In [45]:
def make_spc(spc): 
    """
    make an RMG object from the rms object
    
    """
    if len(spc.adjlist) > 0:
        rmg_spc = Species().from_adjacency_list(spc.adjlist)
    else: 
        rmg_spc = Species().from_smiles(spc.smiles)

    return rmg_spc

In [46]:
sens_times = [1e-2, 3e-2, 100]
sens_rxn_dict = {}
max_len = 0
# get the ordering of reactions that are sensitive to CH3OH at different time scales. 
# our ordering is from highest sensitivity to low. so, start at one. if it is 
# found to be the first most sensitive at the first time value, then the 5th most sensitive at the 
# second time value, then the tenth at the third time value then the score it gets is 
# 1*5*10 = 50

for time in sens_times:
    sens_rxns, rxn_sens = rms.getrxntransitorysensitivities(ssys, "CH3OH", time, tol = 0)
    counter = 1
    for (rxn, sens) in zip(sens_rxns, rxn_sens):
        if rms.getrxnstr(rxn) in sens_rxn_dict.keys():
            old_sens = sens_rxn_dict[rms.getrxnstr(rxn)][0]
            sens_rxn_dict[rms.getrxnstr(rxn)][0] = counter*old_sens
            sens_rxn_dict[rms.getrxnstr(rxn)][1].append(sens)
            sens_rxn_dict[rms.getrxnstr(rxn)][2].append(counter)
        else:
            # print("reac str: ", rms.getrxnstr(rxn))
            # rxn_adj = rms.getrxnadjlist(rxn)
            # rxn_smiles = rms.getrxnsmiles(rxn)
            # print(rxn_smiles)
            reac_spec = [make_spc(reac) for reac in rxn.reactants]
            prod_spec = [make_spc(prod) for prod in rxn.products]
            sens_rxn_dict[rms.getrxnstr(rxn)] = [counter, [sens],[counter], reac_spec, prod_spec]

        counter +=1
    if len(sens_rxns) > max_len:
        max_len = len(sens_rxns)
    
# go through values in sens rxn dict that have fewer than len(sens_times) values 
# and multiply by max_len + 1. 
# penalizing for only being sensitive at one time point
for rxn in sens_rxn_dict.keys():
    if len(sens_rxn_dict[rxn][1]) < len(sens_times):
        sens_rxn_dict[rxn][0] *= (max_len + 1)
        for i in range(len(sens_times) - len(sens_rxn_dict[rxn][1])):
            sens_rxn_dict[rxn][1].append(0)
            sens_rxn_dict[rxn][2].append(max_len + 1)
            
# sort the dictionary by the score
sens_rxn_dict = {k: v for k, v in sorted(sens_rxn_dict.items(), key=lambda item: item[1][0])}

# print the dict: 
for rxn_str, entry in sens_rxn_dict.items():
    print(rxn_str, entry[0])

CH3X+CC(=O)O.[Pt]<=>CH3OH*+CC(=O)[Pt] 3
X+CH3OH*<=>H*+OC[Pt] 4
H*+CH3OH*<=>H2O*+CH3X 32
CH2O*+OC[Pt]<=>X+OCOC[Pt] 36
HCO*+CH3OH*<=>H*+COC=O.[Pt] 125
H*+COC=O.[Pt]<=>X+COCO[Pt] 294
CH3X+OCO.[Pt]<=>OC[Pt]+CH3OH* 630
CO*+CH3OH*<=>HCO*+OC[Pt] 648
CO*+CH3OH*<=>COOH*+CH3X 704
HCOOH*+OC[Pt]<=>HCOO*+CH3OH* 726
CO*+CH3OH*<=>OH*+CC(=O)[Pt] 1300
OH*+CC(=O)O.[Pt]<=>COOH*+CH3OH* 1728
CH3OH*+CC[Pt]<=>OC[Pt]+CC.[Pt] 2556
CO*+CH3OH*<=>H*+O=C([Pt])CO 4116
CH2O*+OC[Pt]<=>HCO*+CH3OH* 4225
X+H2O*<=>H*+OH* 4761
OH*+C=C=O.[Pt]<=>X+C=C(O)O[Pt] 4828
OH*+HCOOH*<=>H2O*+COOH* 4970
H2O*+CC[Pt]<=>OH*+CC.[Pt] 5396
OH*+C=C=O.[Pt]<=>X+O=C([Pt])CO 6674
O*+COOH*<=>OH*+CO2* 7290
OH*+CC=O.[Pt]<=>H2O*+CC(=O)[Pt] 8236
CH2O*+CH3OH*<=>H*+OCOC[Pt] 10584
HCO*+CH3OH*<=>HCOOH*+CH3X 10648
HCOOH*+CC[Pt]<=>HCOO*+CC.[Pt] 10792
H*+CO2*<=>X+COOH* 10800
COOH*+CH3OH*<=>HCOOH*+OC[Pt] 10816
X+HCO*<=>H*+CO* 11322
CH2O*+CC[Pt]<=>HCO*+CC.[Pt] 12638
H2O*+C=C=O.[Pt]<=>H*+C=C(O)O[Pt] 13064
CO*+CC.[Pt]<=>CH3X+CC(=O)[Pt] 13454
H*+CH2O*<=>X+OC[Pt]

In [47]:
# now match up the rms reaction sensitivities with the actual rxn in chemkin
# match species does forward and reverse. if there are multiple matches, it will
# print a warning.
match_list = []
sens_cmkn_dict = {}
for rxn_str, entry in sens_rxn_dict.items():
    for rxn in model.reactions:
        counter = 0
        if rxn.matches_species(entry[3],entry[4]):
            print("match : ", rxn_str, counter)
            match_list.append(rxn_str)
            sens_cmkn_dict[rxn_str] = (entry[0], rxn)
        counter += 1
        if counter >=2: 
            print("more than 1 match found for ", rxn_str)
if len(match_list) == len(sens_rxn_dict.keys()):
    print("all matches found")
else: 
    for rxn_str, entry in sens_rxn_dict.items():
        if rxn_str not in match_list:
            print("no match found for ", rxn_str)

match :  CH3X+CC(=O)O.[Pt]<=>CH3OH*+CC(=O)[Pt] 0
match :  X+CH3OH*<=>H*+OC[Pt] 0
match :  H*+CH3OH*<=>H2O*+CH3X 0
match :  CH2O*+OC[Pt]<=>X+OCOC[Pt] 0
match :  HCO*+CH3OH*<=>H*+COC=O.[Pt] 0
match :  H*+COC=O.[Pt]<=>X+COCO[Pt] 0
match :  CH3X+OCO.[Pt]<=>OC[Pt]+CH3OH* 0
match :  CO*+CH3OH*<=>HCO*+OC[Pt] 0
match :  CO*+CH3OH*<=>COOH*+CH3X 0
match :  HCOOH*+OC[Pt]<=>HCOO*+CH3OH* 0
match :  CO*+CH3OH*<=>OH*+CC(=O)[Pt] 0
match :  OH*+CC(=O)O.[Pt]<=>COOH*+CH3OH* 0
match :  CH3OH*+CC[Pt]<=>OC[Pt]+CC.[Pt] 0
match :  CO*+CH3OH*<=>H*+O=C([Pt])CO 0
match :  CH2O*+OC[Pt]<=>HCO*+CH3OH* 0
match :  X+H2O*<=>H*+OH* 0
match :  OH*+C=C=O.[Pt]<=>X+C=C(O)O[Pt] 0
match :  OH*+HCOOH*<=>H2O*+COOH* 0
match :  H2O*+CC[Pt]<=>OH*+CC.[Pt] 0
match :  OH*+C=C=O.[Pt]<=>X+O=C([Pt])CO 0
match :  O*+COOH*<=>OH*+CO2* 0
match :  OH*+CC=O.[Pt]<=>H2O*+CC(=O)[Pt] 0
match :  CH2O*+CH3OH*<=>H*+OCOC[Pt] 0
match :  HCO*+CH3OH*<=>HCOOH*+CH3X 0
match :  HCOOH*+CC[Pt]<=>HCOO*+CC.[Pt] 0
match :  H*+CO2*<=>X+COOH* 0
match :  COOH*+CH

In [48]:
min_key = min(sens_cmkn_dict, key=sens_cmkn_dict.get)
most_sens_rxn = sens_cmkn_dict[min_key][1]

In [49]:
most_sens_rxn.kinetics

SurfaceArrhenius(A=(4.18e+21,'cm^2/(mol*s)'), n=0, Ea=(9.084,'kcal/mol'), T0=(1,'K'), comment="""Estimated using template [Donating;Abstracting] for rate rule [C-OH;*C-3R]
Euclidian distance = 3.605551275463989
family: Surface_Abstraction_Single_vdW""")

In [49]:
most_sens_rxn_copy = copy.deepcopy(sens_cmkn_dict)
most_sens_rxn_copy.pop(min_key)

try: 
    popped_key = most_sens_rxn_copy[min_key]
except KeyError:
    print("key not found")

key not found


In [50]:
for rxn_str, entry in sens_cmkn_dict.items():
    print(rxn_str, entry[0], entry[1])

CH3X+CC(=O)O.[Pt]<=>CH3OH*+CC(=O)[Pt] 3 CH3X(33) + CC(=O)O.[Pt](110) <=> CH3OH*(23) + CC(=O)[Pt](97)
X+CH3OH*<=>H*+OC[Pt] 4 X(1) + CH3OH*(23) <=> H*(10) + OC[Pt](49)
H*+CH3OH*<=>H2O*+CH3X 32 H*(10) + CH3OH*(23) <=> H2O*(13) + CH3X(33)
CH2O*+OC[Pt]<=>X+OCOC[Pt] 36 CH2O*(20) + OC[Pt](49) <=> X(1) + OCOC[Pt](115)
HCO*+CH3OH*<=>H*+COC=O.[Pt] 125 HCO*(16) + CH3OH*(23) <=> H*(10) + COC=O.[Pt](62)
H*+COC=O.[Pt]<=>X+COCO[Pt] 294 H*(10) + COC=O.[Pt](62) <=> X(1) + COCO[Pt](76)
CH3X+OCO.[Pt]<=>OC[Pt]+CH3OH* 630 CH3X(33) + OCO.[Pt](82) <=> OC[Pt](49) + CH3OH*(23)
CO*+CH3OH*<=>HCO*+OC[Pt] 648 CO*(14) + CH3OH*(23) <=> HCO*(16) + OC[Pt](49)
CO*+CH3OH*<=>COOH*+CH3X 704 CO*(14) + CH3OH*(23) <=> COOH*(18) + CH3X(33)
HCOOH*+OC[Pt]<=>HCOO*+CH3OH* 726 HCOOH*(19) + OC[Pt](49) <=> HCOO*(17) + CH3OH*(23)
CO*+CH3OH*<=>OH*+CC(=O)[Pt] 1300 CO*(14) + CH3OH*(23) <=> OH*(12) + CC(=O)[Pt](97)
OH*+CC(=O)O.[Pt]<=>COOH*+CH3OH* 1728 OH*(12) + CC(=O)O.[Pt](110) <=> COOH*(18) + CH3OH*(23)
CH3OH*+CC[Pt]<=>OC[Pt]+CC.[Pt] 2

In [38]:
if most_sens_rxn.kinetics.comment.startswith("Estimated"):
    print("most sensitive reaction is an estimate")

most sensitive reaction is an estimate


In [51]:
# get the specific node for the sensitive reaction. find out the highest populated parent node 

# load the kinetics database
RMG_db_folder= "/Users/blais.ch/Documents/_01_code/RMG_env_1/RMG-database/"
# Specify the path to the families
families_dir = os.path.join(RMG_db_folder,"input","kinetics","families")
if not os.path.exists(families_dir):
    raise OSError(f'Path to rules does not exist:\n{families_dir}')

# Specify the path to the libraries
kinetic_libraries_dir = os.path.join(RMG_db_folder,"input","kinetics","libraries","Surface")
if not os.path.exists(kinetic_libraries_dir):
    raise OSError(f'Path to kinetic libraries does not exist:\n{kinetic_libraries_dir}')

# do not load training, this will create more rules. in future, may want to 
# create those rules for perturbation, not sure. 
kinetics_families = ['all']
kinetics_database = KineticsDatabase()
kinetics_database.load_families(
        path=families_dir,
        families=kinetics_families,)

In [73]:
# breadcrumb cjb need to determine which sensitive reactions we should make rules for. 
# start with only one. add a test for most sensitive reaction, if a rule already exists, go for second most sensitive.

from rmgpy.data.kinetics.family import TemplateReaction
# test for reaction: get the nodes that would generate this rxn
rxn = model.reactions[30]
family = rxn.get_source()
template = rxn.template

# kinetics_database.families[family].get_labeled_reactants_and_products(rxn.reactants, rxn.products)
kinetics_database.families[family].add_atom_labels_for_reaction(rxn)
template = kinetics_database.families[family].get_reaction_template(rxn)
# kinetics_database.families[family].add_entry()
# kinetics_database.families[family].has_rate_rule(template)
source_info = kinetics_database.families[family].extract_source_from_comments(rxn)
# new_rule = kinetics_database.families[family].get_kinetics_for_template(template)
parent = source_info[1][1]["rules"][0][0]
grp = rxn
name = 'O-C;VacantSite1;VacantSite2'
# kinetics_database.families[family].add_entry(parent, grp, name)

rank = parent.rank + 1

In [62]:
kinetics_database.families[family].rules.entries["H2;VacantSite1;VacantSite2"][0].index

2

In [72]:
for entry in kinetics_database.families[family].rules.entries:
    index = kinetics_database.families[family].rules.entries[entry][0].index
    print(type(entry))
index

<class 'str'>
<class 'str'>


2

In [63]:
last_index = 0
for entry in kinetics_database.families[family].rules.entries: 
    index = entry[0].index
    if index > last_index:

        last_index = index
new_index = last_index + 1
new_index


TypeError: '>' not supported between instances of 'builtin_function_or_method' and 'int'

In [24]:

data = model.reactions[30].kinetics

if isinstance(data, StickingCoefficient):
    data = StickingCoefficientBEP(
        # todo: perhaps make a method StickingCoefficient.StickingCoefficientBEP
        #  analogous to Arrhenius.to_arrhenius_ep
        A=deepcopy(data.A),
        n=deepcopy(data.n),
        alpha=0,
        E0=deepcopy(data.Ea),
        Tmin=deepcopy(data.Tmin),
        Tmax=deepcopy(data.Tmax),
        coverage_dependence=deepcopy(data.coverage_dependence),
    )
elif isinstance(data, SurfaceArrhenius):
    data = SurfaceArrheniusBEP(
        # todo: perhaps make a method SurfaceArrhenius.toSurfaceArrheniusBEP
        #  analogous to Arrhenius.to_arrhenius_ep
        A=deepcopy(data.A),
        n=deepcopy(data.n),
        alpha=0,
        E0=deepcopy(data.Ea),
        Tmin=deepcopy(data.Tmin),
        Tmax=deepcopy(data.Tmax),
        coverage_dependence=deepcopy(data.coverage_dependence),
    )
data

StickingCoefficientBEP(A=0.1, n=0, alpha=0, E0=(6.912,'kcal/mol'))

In [25]:
from rmgpy.data.base import Entry
from rmgpy.reaction import Reaction

index = 3
data = data

new_entry = Entry(
    index=index,
    label=';'.join([g.label for g in template]),
    item=Reaction(reactants=[g.item for g in template], products=[]),
    data=data,
    rank=rank,
    short_desc="Rate rule generated for uncertainty",
    long_desc="Rate rule generated for uncertainty",
)
# new_entry.data.comment = "From training reaction {1} used for {0}".format(
#     ';'.join([g.label for g in template]), entry.index)

# new_entry.data.A.value_si /= entry.item.degeneracy
try:
    kinetics_database.families[family].rules.entries[new_entry.label].append(new_entry)
except KeyError:
    print("key already exists, updating entry")
    kinetics_database.families[family].rules.entries[new_entry.label] = [new_entry]
# index += 1

In [26]:
kinetics_database.families[family].rules.entries

OrderedDict([('Adsorbate;VacantSite1;VacantSite2',
              [<Entry index=1 label="Adsorbate;VacantSite1;VacantSite2">]),
             ('H2;VacantSite1;VacantSite2',
              [<Entry index=2 label="H2;VacantSite1;VacantSite2">]),
             ('O-C;VacantSite1;VacantSite2',
              [<Entry index=3 label="O-C;VacantSite1;VacantSite2">])])

In [27]:

kinetics_database.families[family].rules.save(os.path.join(families_dir, family, 'rules_' + 'test' + '.py'))

In [28]:
kinetics_database.families[family].groups.entries

OrderedDict([('Adsorbate', <Entry index=1 label="Adsorbate">),
             ('VacantSite1', <Entry index=2 label="VacantSite1">),
             ('VacantSite2', <Entry index=3 label="VacantSite2">),
             ('H2', <Entry index=4 label="H2">),
             ('O', <Entry index=5 label="O">),
             ('O-H', <Entry index=6 label="O-H">),
             ('H2O', <Entry index=7 label="H2O">),
             ('O-N', <Entry index=9 label="O-N">),
             ('O-C', <Entry index=10 label="O-C">),
             ('N', <Entry index=11 label="N">),
             ('N-C', <Entry index=12 label="N-C">),
             ('N-H', <Entry index=13 label="N-H">),
             ('C-H', <Entry index=14 label="C-H">),
             ('Adsorbed1', <Entry index=-1 label="Adsorbed1">),
             ('Adsorbed2', <Entry index=-1 label="Adsorbed2">)])

In [29]:
for i in dir(model.reactions[30]):
    if "Reaction" in i:
        print(i)

In [3]:
chemkin_surf_file


NameError: name 'chemkin_surf_file' is not defined

In [18]:
with open(chemkin_surf_file, 'r') as f:
    chemkin_surf_file_lines = f.readlines()
    for line in chemkin_surf_file_lines:
        if line.startswith("SITE"):
            site_density = float(line.split("/")[1])
            site_density = site_density * 1e4 # convert from mol/cm2 to mol/m2
            break

['SITE   SDEN', '2.9430E-09', ' ! mol', 'cm^2\n']
2.9429999999999998e-05


In [21]:
from sbr import rms_sbr


In [1]:
import yaml
from sbr import rms_sbr

# testing for local computer before using discovery
file_path = "/Users/blais.ch/Documents/_01_code/05_Project_repos_Github/meOH_repos/uncertainty_analysis/rmg_gua/baseline/rms/chem53.rms"
rmg_db_folder = "/Users/blais.ch/Documents/_01_code/RMG_env_1/RMG-database/"
expt_condts = "/Users/blais.ch/Documents/_01_code/05_Project_repos_Github/meOH_repos/uncertainty_analysis/rmg_gua/gua_cantera/all_experiments_reorg_sbr.yaml"

with open(expt_condts, 'r') as file:
    data = yaml.safe_load(file)

# pick just one experiment for example
conditions = data[2]

test_sbr = rms_sbr(
    file_path,
    reac_config=conditions,
    rtol=1.0e-11,
    atol=1.0e-22,
)
results = test_sbr.run_simulation()


In [5]:
for key, val in test_sbr.graaf_mole_fracs.items(): 
    results["graaf moles" + key] = val

In [7]:
species = list(test_sbr.graaf_mole_fracs.keys())

In [15]:
# calculate MSE
for spec in species:
    results["MSE " + spec] = abs(results["graaf moles" + spec] - 
    results[spec])/results["graaf moles" + spec]
    print("MSE for " + spec + " is " + str(results["MSE " + spec]), " Graaf: ", results["graaf moles" + spec], " RMG: ", results[spec])

MSE for CH3OH is 0.9999741013209418  Graaf:  0.0078  RMG:  2.020096966544339e-07
MSE for CO is 0.4553622480083891  Graaf:  0.067  RMG:  0.09750927061656207
MSE for CO2 is 0.44244377604817536  Graaf:  0.2538  RMG:  0.1415077696389731
MSE for H2 is 0.9544735216958216  Graaf:  0.6636  RMG:  0.03021137100265276
MSE for H2O is 63.07990261406577  Graaf:  0.0078  RMG:  0.499823240389713


In [1]:
import pickle

with open("../baseline/sens_cmkn_dict.pickle", "rb") as f:
    sens_dict = pickle.load(f)
sens_dict


{'CH2O*+CH3O*<=>HCO*+CH3OH*': (2,
  TemplateReaction(index=104, reactants=[Species(index=20, label="CH2O*", thermo=NASA(polynomials=[NASAPolynomial(coeffs=[-1.46696,0.041387,-8.87611e-05,9.10103e-08,-3.36603e-11,-20300.2,15.7178], Tmin=(100,'K'), Tmax=(879.32,'K')), NASAPolynomial(coeffs=[-0.892954,0.0166197,-8.71599e-06,1.66817e-09,-1.12505e-13,-19544.6,17.893], Tmin=(879.32,'K'), Tmax=(5000,'K'))], Tmin=(100,'K'), Tmax=(5000,'K'), comment="""Thermo library: surfaceThermoPt111 Binding energy corrected by LSR () from Pt111"""), molecule=[Molecule(smiles="C=O.[Pt]")], molecular_weight=(30.026,'amu')), Species(index=21, label="CH3O*", thermo=NASA(polynomials=[NASAPolynomial(coeffs=[-1.68422,0.0428024,-7.50785e-05,7.03684e-08,-2.50921e-11,-22313.7,7.76424], Tmin=(100,'K'), Tmax=(857.66,'K')), NASAPolynomial(coeffs=[1.14794,0.0171345,-8.39641e-06,1.59809e-09,-1.08976e-13,-22341.3,-2.79304], Tmin=(857.66,'K'), Tmax=(5000,'K'))], Tmin=(100,'K'), Tmax=(5000,'K'), comment="""Thermo library: su

In [4]:
for key, val in sens_dict.values():
    print(key, val[2])

TypeError: 'TemplateReaction' object is not subscriptable

In [27]:
# min_key = min(sens_dict, key=sens_dict[0].get)
most_sens_id = min(sens_dict.items(), key=lambda k: k[0][0])[0]
sens_dict[most_sens_id][1].family

'Surface_Abstraction_Single_vdW'

In [ ]:
# open the chemkin sensitivity file 
with open(